In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=7)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,21 mins 19 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_edu_r8114u
H2O cluster total nodes:,1
H2O cluster free memory:,990 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,7
H2O cluster status:,"locked, healthy"


In [3]:
train_high = h2o.import_file('../cleaned-data/04-train_high-catAsInt-withXYZ.csv')
train_low = h2o.import_file('../cleaned-data/04-train_low-catAsInt-withXYZ.csv')
test_high = h2o.import_file('../cleaned-data/04-test_high-catAsInt-withXYZ.csv')
test_low = h2o.import_file('../cleaned-data/04-test_low-catAsInt-withXYZ.csv')      
train_high.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


id,carat,cut,color,clarity,depth,table,x,y,z,price
8,2.05,4,6,1,62.4,57,8.17,8.09,5.08,18193
34,2.01,0,1,4,64.9,60,7.82,7.78,5.06,13744
42,2,1,2,3,64.2,52,7.91,7.88,5.07,11400
72,2,1,7,3,60.3,56,8.08,8.11,4.88,16425
82,2.28,4,1,4,62.7,57,8.43,8.36,5.26,16450
93,2.02,2,2,1,60.4,58,8.2,8.16,4.94,17357
141,2,3,3,3,60.7,60,8.07,8.11,4.91,15312
154,2.01,3,2,0,61.5,62,8.07,7.94,4.92,15475
165,2.31,3,2,3,62.9,58,8.43,8.34,5.27,17218
167,2.01,4,3,1,60.6,57,8.19,8.09,4.92,17383


In [4]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

(1615, 11)
(38730, 11)
(521, 10)
(12928, 10)


[None, None, None, None]

In [5]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']

In [6]:
train_frame = train_high
model_name = 'diamonds_high'
runtime = 3540

aml_high = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

AutoML progress: |
14:53:57.352: New models will be added to existing leaderboard diamonds_high@@price (leaderboard frame=null) with already 66 models.

████████████████████████████████████████████████████████| 100%

15:54:22.777: StackedEnsemble_BestOfFamily_AutoML_20200301_145357 [StackedEnsemble best (built using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
15:54:23.779: StackedEnsemble_AllModels_AutoML_20200301_145357 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
16912.2
16459.6
16056.5
17029
15680.3
13442.7
16592.6
16900.9
15108.7
15600.8


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20200301_145357,2.82563e+06,1680.96,2.82563e+06,1323.02,0.120789
XGBoost_3_AutoML_20200301_145357,2.84805e+06,1687.62,2.84805e+06,1340.91,0.120861
GBM_4_AutoML_20200301_145357,2.86207e+06,1691.76,2.86207e+06,1324.68,0.12116
GBM_grid__1_AutoML_20200301_145357_model_26,2.86458e+06,1692.51,2.86458e+06,1344.39,0.121908
XGBoost_1_AutoML_20200301_145357,2.87152e+06,1694.55,2.87152e+06,1327.06,0.121378
XGBoost_2_AutoML_20200301_145357,2.87222e+06,1694.76,2.87222e+06,1324.59,0.121121
GBM_grid__1_AutoML_20200301_145357_model_36,2.87278e+06,1694.93,2.87278e+06,1328.35,0.122212
XGBoost_grid__1_AutoML_20200301_145357_model_10,2.87705e+06,1696.19,2.87705e+06,1339.96,0.121893
GBM_1_AutoML_20200301_145357,2.8777e+06,1696.38,2.8777e+06,1329.94,0.122071
GBM_3_AutoML_20200301_145357,2.87771e+06,1696.38,2.87771e+06,1330.09,0.122157


In [7]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [ ]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

In [ ]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [ ]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

In [ ]:
h2o.export_file(frame=combinedTable,path='../predictions/14-Auto-ML-Split-data-catAsInt-withXYZ.csv')